In [42]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Remplacez "CLIENT_ID" et "CLIENT_SECRET" par vos propres identifiants de l'application Spotify
client_id = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"

# Créez une instance de l'authentification client-credentials pour utiliser l'API Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Recherchez des morceaux correspondant à une requête spécifique
results = sp.search(q='artist:Ed Sheeran track:Shape of You', type='track')

# Affichez le nom de chaque morceau correspondant à la requête
for track in results['tracks']['items']:
    print(track['name'])

Shape of You
Shape of You
Shape of You - Acoustic
Shape of You (feat. Zion & Lennox) - Latin Remix
Shape of You (feat. Nyla & Kranium) - Major Lazer Remix
Shape of You - Galantis Remix
Shape of You - Stormzy Remix
Shape of You - NOTD Remix
Shape of You - Yxng Bane Remix
Shape of You


In [43]:

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

track_id = 'spotify:track:6rqhFgbbKwnb9MLmUQDhG6' # Example track ID
track_features = sp.audio_features(track_id)
print(track_features)

[{'danceability': 0.592, 'energy': 0.0196, 'key': 1, 'loudness': -33.35, 'mode': 1, 'speechiness': 0.0358, 'acousticness': 0.362, 'instrumentalness': 0.854, 'liveness': 0.108, 'valence': 0.0312, 'tempo': 119.504, 'type': 'audio_features', 'id': '6rqhFgbbKwnb9MLmUQDhG6', 'uri': 'spotify:track:6rqhFgbbKwnb9MLmUQDhG6', 'track_href': 'https://api.spotify.com/v1/tracks/6rqhFgbbKwnb9MLmUQDhG6', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6rqhFgbbKwnb9MLmUQDhG6', 'duration_ms': 65314, 'time_signature': 4}]


In [44]:
import base64
import requests


client_ID = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"

# encode client_id and client_secret in base64
client_credentials = f"{client_id}:{client_secret}".encode("ascii")
base64_credentials = base64.b64encode(client_credentials).decode("ascii")

# make request for access token
url = "https://accounts.spotify.com/api/token"
response = requests.post(
    url,
    headers={
        "Authorization": f"Basic {base64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data={"grant_type": "client_credentials"}
)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print(f"Access Token: {access_token}")
else:
    print(f"Error: {response.text}")

Access Token: BQBF0M-NeGzFHGfcrZIZ44dwiq3uxBhqEfG4ISgAZBW5lpn6mXom7Ucbj9w6loQgMk0gkdeeh0qC6jwWC5y86PKP3e9GCOANyuu0Hr17NOLDeQ9UDq4r


In [45]:
import pandas as pd
import requests
from tqdm import tqdm

client_id = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"


# Obtenez les caractéristiques musicales et la popularité de chaque piste
def get_audio_features(track_id):
    global access_token
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    response = requests.get(
        url,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
    )
    if response.status_code == 200:
        data = response.json()
        return [
            data["danceability"],
            data["energy"],
            data["key"],
            data["loudness"],
            data["mode"],
            data["speechiness"],
            data["acousticness"],
            data["instrumentalness"],
            data["liveness"],
            data["valence"],
            data["tempo"],
            data["duration_ms"]
        ]
    else:
        return None

# Requêtez l'API Spotify pour obtenir les pistes sorties en 2022
def get_new_tracks():
    global access_token
    tracks = []
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": "year:2022",
        "type": "track",
        "market": "US",
        "limit": 50,
        "offset": 0
    }
    while True:
        response = requests.get(
            url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {access_token}"
            },
            params=params
        )
        if response.status_code == 200:
            data = response.json()
            tracks += data["tracks"]["items"]
            if data["tracks"]["next"] is None:
                break
            params["offset"] += 50
        else:
            print("Erreur lors de la récupération des pistes : ", response.text)
            break
    return tracks

# Récupérez les pistes et leurs caractéristiques musicales
def get_tracks_audio_features():
    global access_token
    tracks = get_new_tracks()
    audio_features = []
    for track in tqdm(tracks):
        track_info = {
            "id": track["id"],
            "name": track["name"],
            "album": track["album"]["name"],
            "artists": [artist["name"] for artist in track["artists"]],
            "popularity": track["popularity"],
            "explicit": track["explicit"],
            "release_date": track["album"]["release_date"],
            "duration_ms": track["duration_ms"]
        }
        features = get_audio_features(track["id"])
        if features is not None:
            track_info.update({
                "acousticness": features[6],
                "danceability": features[0],
                "energy": features[1],
                "instrumentalness": features[7],
                "key": features[2],
                "liveness": features[8],
                "loudness": features[3],
                "mode": features[4],
                "speechiness": features[5],
                "tempo": features[10],
                "valence": features[9]
            })
            audio_features.append(track_info)
    return pd.DataFrame(audio_features)

df = get_tracks_audio_features()
print(df.head())

100%|██████████| 1000/1000 [01:06<00:00, 15.01it/s]

                       id                                    name  \
0  3OHfY25tqY28d16oZczHc8                               Kill Bill   
1  2dHHgzDwk4BJdRwy9uXhTO  Creepin' (with The Weeknd & 21 Savage)   
2  1Qrg8KqiBpW07V7PNxwwwL                               Kill Bill   
3  1bDbXMyjaUIooNwFE9wn0N                               Rich Flex   
4  4FyesJzVpA39hbYvcseO2d                         Just Wanna Rock   

               album                                artists  popularity  \
0                SOS                                  [SZA]          92   
1  HEROES & VILLAINS  [Metro Boomin, The Weeknd, 21 Savage]          93   
2                SOS                                  [SZA]          92   
3           Her Loss                     [Drake, 21 Savage]          89   
4    Just Wanna Rock                         [Lil Uzi Vert]          88   

   explicit release_date  duration_ms  acousticness  danceability  energy  \
0     False   2022-12-09       153946        0.0543      

In [46]:
df

,id,name,album,artists,popularity,explicit,release_date,duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,3OHfY25tqY28d16oZczHc8,Kill Bill,SOS,[SZA],92,False,2022-12-09,153946,0.054300,0.644,0.728,0.169000,8,0.1610,-5.750,1,0.0351,88.993,0.4300
1,2dHHgzDwk4BJdRwy9uXhTO,Creepin' (with The Weeknd & 21 Savage),HEROES & VILLAINS,"[Metro Boomin, The Weeknd, 21 Savage]",93,True,2022-12-02,221520,0.417000,0.715,0.620,0.000000,1,0.0822,-6.005,0,0.0484,97.950,0.1720
2,1Qrg8KqiBpW07V7PNxwwwL,Kill Bill,SOS,[SZA],92,False,2022-12-08,153946,0.052100,0.644,0.735,0.144000,8,0.1610,-5.747,1,0.0391,88.980,0.4180
3,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,Her Loss,"[Drake, 21 Savage]",89,True,2022-11-04,239359,0.050300,0.561,0.520,0.000002,11,0.3550,-9.342,0,0.2440,153.150,0.4240
4,4FyesJzVpA39hbYvcseO2d,Just Wanna Rock,Just Wanna Rock,[Lil Uzi Vert],88,True,2022-10-17,123890,0.065200,0.486,0.545,0.004740,11,0.0642,-7.924,1,0.0336,150.187,0.0385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,1H5Mpak0oTTIGmQ2LrXlaM,Ballin,Ballin,[Mike Sherm],61,True,2022-05-13,125586,0.025600,0.971,0.509,0.000000,1,0.1030,-9.752,1,0.3420,106.977,0.4260
698,3OM6qQmdFV6uy61GIqpRtf,AMAZING,WHO CARES?,[Rex Orange County],72,False,2022-03-11,209200,0.279000,0.719,0.461,0.000000,8,0.0875,-8.191,1,0.0454,123.915,0.5480
699,7wdwIaXUuzlu1grzWMFRJm,Mary On A Cross,[MESSAGE FROM THE CLERGY],[Ghost],70,False,2022-09-12,244804,0.000187,0.467,0.900,0.000493,11,0.1050,-4.505,1,0.0451,130.027,0.5540
700,3qKunud9sySc8qvQ3i84og,Conceited,"You Still Here, Ho ?",[Flo Milli],63,True,2022-07-20,146854,0.056600,0.984,0.719,0.000099,8,0.5170,-5.119,1,0.0893,118.015,0.4370


In [47]:
import pandas as pd
import numpy as np


# Calculate some statistics on the data
mean_liveness = np.mean(df['liveness'])
max_duration = np.max(df['duration_ms'])

print('Mean liveness: ', mean_liveness)
print('Max duration: ', max_duration)

Mean liveness:  0.1830948717948718
Max duration:  481802


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                702 non-null    object 
 1   name              702 non-null    object 
 2   album             702 non-null    object 
 3   artists           702 non-null    object 
 4   popularity        702 non-null    int64  
 5   explicit          702 non-null    bool   
 6   release_date      702 non-null    object 
 7   duration_ms       702 non-null    int64  
 8   acousticness      702 non-null    float64
 9   danceability      702 non-null    float64
 10  energy            702 non-null    float64
 11  instrumentalness  702 non-null    float64
 12  key               702 non-null    int64  
 13  liveness          702 non-null    float64
 14  loudness          702 non-null    float64
 15  mode              702 non-null    int64  
 16  speechiness       702 non-null    float64
 1

In [56]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Define X and y
X = df.drop(['popularity'], axis=1)
y = df['popularity']

# Convert the list column to string column
X['artists'] = X['artists'].apply(lambda x: ','.join(map(str, x)))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Preprocess the training data
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_cols),
                                                ('cat', cat_transformer, cat_cols)])

X_train = preprocessor.fit_transform(X_train)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Preprocess the test data
X_test['artists'] = X_test['artists'].apply(lambda x: ','.join(map(str, x)))
X_test = preprocessor.transform(X_test)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

MAE: 4.578561680055512


In [57]:
import mlflow
import mlflow.sklearn

# Assume you have already trained and evaluated a model
with mlflow.start_run() as run:
    # Log the model parameters and performance metrics
    mlflow.log_param('model_type', 'linear_regression')
    mlflow.log_param('num_features', X.shape[1])
    mlflow.log_metric('MAE', mae)

    # Log the trained model as an artifact
    mlflow.sklearn.log_model(model, 'model')
